In [1]:
#install hugging face transformers and torch
!pip install transformers
!pip install torch

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np

#import pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

#import pretrained model
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [3]:
#import csv and sample => CHANGE PATH
headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
headlines_df = headlines_df.sample(300, random_state=42)

In [6]:
#transform to numpy - shuffle and put only headlines in list
headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

headlines_list[:5]

['REMINDER: Alexion Pharmaceuticals Has PDUFA Date For Eculizumab sBLA Today',
 'Astro-Med Q2 EPS $0.17 Up From $0.11 YoY, Sales $33.8M Up From $27.483M YoY',
 'Stocks That Hit 52-Week Lows On Thursday',
 'Stocks That Hit 52-Week Highs On Tuesday',
 'Novabay Pharmaceuticals Chiarman Najafi Buys 50,000 Shares @$0.65 -Form 4']

In [7]:
#tokenize inputs
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')


In [10]:
inputs.shape()

AttributeError: 

In [14]:
#generate output model
outputs = model(**inputs)
print(outputs.logits)

tensor([[-1.2894e-01, -2.0035e+00,  2.3312e+00],
        [-6.7542e-01, -1.0647e+00,  2.4427e+00],
        [-5.4200e-02, -1.7218e+00,  1.7962e+00],
        [-2.7527e-02,  3.8342e-01, -1.0451e+00],
        [-1.3014e+00, -2.5441e-01,  2.0419e+00],
        [ 1.3989e+00, -1.1835e+00, -5.7344e-01],
        [-6.1789e-01, -6.1832e-01,  2.2074e+00],
        [-4.1380e-01, -1.6176e+00,  2.3685e+00],
        [-1.5047e+00,  3.6868e-01,  1.9665e+00],
        [-6.3233e-01, -1.2856e+00,  2.3468e+00],
        [ 1.4522e+00, -1.9045e+00, -5.5520e-01],
        [-6.2040e-01, -4.7357e-01,  1.6303e+00],
        [-5.7356e-01, -1.7327e+00,  2.5544e+00],
        [-3.3227e-01, -8.8253e-01,  1.5452e+00],
        [-1.5273e+00,  4.1823e-01,  2.0009e+00],
        [-1.8193e+00,  2.9770e+00, -8.1709e-01],
        [-9.6787e-01,  1.1750e+00, -4.0121e-02],
        [-1.6514e+00,  1.5297e+00,  8.9895e-01],
        [-1.2285e+00,  2.8388e+00, -1.6165e+00],
        [-4.5131e-02, -1.4216e+00,  1.4730e+00],
        [ 4.4266e-01

In [16]:
#translate outputs to predictions via softmax
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0778, 0.0119, 0.9103],
        [0.0412, 0.0279, 0.9309],
        [0.1324, 0.0250, 0.8426],
        [0.3485, 0.5256, 0.1260],
        [0.0311, 0.0886, 0.8803],
        [0.8232, 0.0622, 0.1145],
        [0.0530, 0.0530, 0.8940],
        [0.0573, 0.0172, 0.9255],
        [0.0252, 0.1640, 0.8108],
        [0.0472, 0.0246, 0.9283],
        [0.8553, 0.0298, 0.1149],
        [0.0858, 0.0994, 0.8148],
        [0.0414, 0.0130, 0.9456],
        [0.1232, 0.0711, 0.8057],
        [0.0238, 0.1664, 0.8099],
        [0.0080, 0.9702, 0.0218],
        [0.0830, 0.7072, 0.2098],
        [0.0264, 0.6354, 0.3382],
        [0.0166, 0.9721, 0.0113],
        [0.1719, 0.0434, 0.7847],
        [0.2074, 0.0196, 0.7730],
        [0.0285, 0.9485, 0.0230],
        [0.0474, 0.2577, 0.6949],
        [0.9272, 0.0358, 0.0370],
        [0.8819, 0.0189, 0.0993],
        [0.7062, 0.0365, 0.2573],
        [0.0244, 0.0391, 0.9365],
        [0.0970, 0.0873, 0.8157],
        [0.0085, 0.9736, 0.0179],
        [0.745

In [17]:
#predictions to df
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [18]:
#sorted df
df.sort_values(by='Positive', ascending=False)

,Headline,Positive,Negative,Neutral
261,"Earnings Preview: Stryker EPS, Revenue Expecte...",0.955498,0.020781,0.023721
211,Asterias Provides 6 Mo. Data Readout From Its ...,0.954589,0.014721,0.030690
94,Raymond James Says Client AUM Grew 4% Y/Y To $...,0.951677,0.016788,0.031535
205,"Courier Corp Holder Gamco Reports 6.97%, Up Fr...",0.951596,0.027925,0.020479
117,SAExploration shares are trading higher after ...,0.950563,0.018216,0.031221
...,...,...,...,...
96,Shares of several oil and gas companies are tr...,0.008023,0.971352,0.020626
15,Optical Names Move Lower in Late-Afternoon Tra...,0.008013,0.970155,0.021831
272,Shares of several healthcare companies are tra...,0.007892,0.973975,0.018133
240,United Air Shares Down 11.4% Following Q4 Earn...,0.007161,0.970784,0.022055
